### Setup

##### Install

In [57]:
%pip install twilio python-dotenv matplotlib numpy diagrams graphviz mermaid-py --quiet

98655.25s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


##### Init Client

##### Functions

In [58]:
import os
from twilio.rest import Client
from matplotlib.patches import FancyBboxPatch, ConnectionPatch
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as patches

import IPython

from mermaid import Mermaid
from IPython.display import display

# Find your Account SID and Auth Token at twilio.com/console
# and set the environment variables. See http://twil.io/secure
account_sid = os.environ["TWILIO_ACCOUNT_SID"]
auth_token = os.environ["TWILIO_AUTH_TOKEN"]
client = Client(account_sid, auth_token)


In [59]:
from IPython.display import display, HTML
import xml.dom.minidom

def pretty_print_twiml(twiml_response):
    """
    Takes a TwiML VoiceResponse object and renders it in a Twilio-branded pretty HTML box.
    
    Args:
        twiml_response: A twilio.twiml.voice_response.VoiceResponse object
    
    Returns:
        str: Pretty formatted XML string (also displayed in a styled box)
    """
    # Convert TwiML response to pretty XML
    xml_string = str(twiml_response)
    pretty_xml = xml.dom.minidom.parseString(xml_string).toprettyxml(indent="  ")
    pretty_xml = '\n'.join(pretty_xml.split('\n')[1:])  # Remove XML declaration

    # Escape HTML special characters
    escaped_xml = pretty_xml.replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;")

    # Twilio-style display box
    style = """
    <style>
    .twilio-xml-box {
        background: #0B0D17;
        border: 1px solid #661847;
        padding: 20px;
        border-radius: 12px;
        font-family: 'Courier New', monospace;
        font-size: 0.95em;
        color: #FFFFFF;
        white-space: pre-wrap;
        line-height: 1.5;
        box-shadow: 0 0 12px rgba(242, 47, 70, 0.4);
        margin: 20px auto;
        width: 90%;
    }
    .twilio-xml-box span.tag {
        color: #F22F46;
    }
    .twilio-xml-box span.attr {
        color: #0DC5FB;
    }
    </style>
    """

    html_output = f"""
    {style}
    <div class="twilio-xml-box">{escaped_xml}</div>
    """

    display(HTML(html_output))
    return pretty_xml.strip()


In [60]:
import time
from IPython.display import display, clear_output, HTML

def animate_call_flow_html(sid=None, delay=1.5):
    steps = [
        "Connecting Call",
        "AI Agent connected",
        "Text to Speech initiated",
        "Call Ended"
    ]

    style = """
    <style>
    @keyframes gradientFlow {
        0% { background-position: 0% 50%; }
        50% { background-position: 100% 50%; }
        100% { background-position: 0% 50%; }
    }
    .twilio-flow {
        font-family: 'Segoe UI', sans-serif;
        font-size: 1.6em;
        color: white;
        font-weight: 600;
        padding: 18px 24px;
        border-radius: 12px;
        text-align: center;
        background: linear-gradient(-45deg, #0DC5FB, #661847, #4B0082, #F22F46);
        background-size: 400% 400%;
        animation: gradientFlow 6s ease infinite;
        box-shadow: 0 0 20px rgba(0,0,0,0.3);
        margin: 20px auto;
        width: 70%;
    }
    .twilio-sid {
        font-size: 0.95em;
        margin-top: 10px;
        color: #888;
    }
    </style>
    """

    flow = ""
    for step in steps:
        flow += f"{step} ➝ "
        clear_output(wait=True)
        html = f"""
        {style}
        <div class="twilio-flow">{flow.strip(' ➝ ')}</div>
        """
        display(HTML(html))
        time.sleep(delay)

    if sid:
        final = f"""
        {style}
        <div class="twilio-flow">Call Successful!</div>
        <div class="twilio-sid">Call SID: <code>{sid}</code></div>
        """
        clear_output(wait=True)
        display(HTML(final))


In [61]:
from IPython.display import display, HTML

def pretty_print_operator_results(transcripts):
    from html import escape

    style = """
    <style>
    .twilio-box {
        background: #0B0D17;
        border: 1px solid #661847;
        border-radius: 12px;
        color: #ffffff;
        font-family: 'Segoe UI', sans-serif;
        padding: 20px;
        margin: 20px auto;
        width: 95%;
        box-shadow: 0 0 20px rgba(242, 47, 70, 0.2);
    }
    .twilio-header {
        font-size: 1.1em;
        font-weight: bold;
        margin-bottom: 12px;
        color: #F22F46;
    }
    .twilio-sub {
        color: #0DC5FB;
        font-size: 0.9em;
        margin-bottom: 10px;
    }
    .twilio-block {
        background: #12284C;
        padding: 16px;
        border-radius: 8px;
        margin-bottom: 20px;
        border-left: 4px solid #4B0082;
        font-size: 0.9em;
    }
    .twilio-key {
        font-weight: bold;
        color: #FFFFFF;
    }
    .twilio-value {
        color: #BBBBBB;
    }
    .twilio-divider {
        margin: 16px 0;
        border-top: 1px solid #333;
    }
    </style>
    """

    html = f"""
    {style}
    <div class="twilio-box">
        <div class="twilio-header">Twilio Intelligence Operator Results</div>
        <div class="twilio-sub">Getting operator results for {len(transcripts)} transcripts.</div>
    """

    for transcript in transcripts:
        html += f"""<div class="twilio-block">
        <div class="twilio-key">Transcript SID:</div>
        <div class="twilio-value">{escape(transcript.sid)}</div>
        """

        try:
            operator_results = client.intelligence.v2.transcripts(
                transcript.sid
            ).operator_results.list(limit=100)

            html += f"""<div class="twilio-sub">Found {len(operator_results)} operator results.</div>"""

            for i, record in enumerate(operator_results, 1):
                html += f"""<div class="twilio-divider"></div>
                <div class="twilio-key">Operator #{i}</div>
                <div class="twilio-value">Name: {record.name}</div>
                <div class="twilio-value">Type: {record.operator_type}</div>
                <div class="twilio-value">Operator SID: {record.operator_sid}</div>
                """

                if record.operator_type == 'text-generation' and record.text_generation_results:
                    result_text = record.text_generation_results.get('result', 'No result available')
                    html += f"""<div class="twilio-key" style="margin-top: 10px;">Text Generation Result:</div>
                                <div class="twilio-value">{escape(result_text)}</div>"""

                elif record.operator_type == 'extract' and record.extract_results:
                    html += "<div class='twilio-key' style='margin-top: 10px;'>Extraction Results:</div>"
                    for entity_type, entities in record.extract_results.items():
                        html += f"<div class='twilio-value'>{entity_type}: {entities}</div>"

                    if record.extract_match:
                        html += f"<div class='twilio-value'>Match Found: {record.extract_match}</div>"
                        html += f"<div class='twilio-value'>Match Probability: {record.match_probability}</div>"

                    if record.utterance_results:
                        html += "<div class='twilio-key' style='margin-top: 10px;'>Utterance Analysis:</div>"
                        for j, utterance in enumerate(record.utterance_results):
                            html += f"<div class='twilio-sub'>Utterance {j+1} (Index {utterance['utterance_index']}):</div>"
                            html += f"<div class='twilio-value'>Match Probability: {utterance['match_probability']}</div>"
                            labeled_text = ""
                            for part in utterance['utterance_parts']:
                                label = part.get('label')
                                text = part['text']
                                labeled_text += f"[{label}: {text}]" if label else text
                            html += f"<div class='twilio-value'>Text: {escape(labeled_text)}</div>"

                elif record.operator_type == 'conversation-classify':
                    html += "<div class='twilio-key' style='margin-top: 10px;'>Classification Result:</div>"
                    html += f"<div class='twilio-value'>Predicted Label: {record.predicted_label}</div>"
                    html += f"<div class='twilio-value'>Predicted Probability: {record.predicted_probability}</div>"

                    if record.label_probabilities:
                        html += "<div class='twilio-key'>All Label Probabilities:</div>"
                        for label, prob in record.label_probabilities.items():
                            html += f"<div class='twilio-value'>{label}: {prob}</div>"

                html += f"<div class='twilio-value' style='margin-top: 10px;'>Transcript SID: {record.transcript_sid}</div>"
                html += f"<div class='twilio-value'>URL: <a style='color:#0DC5FB;' href='{record.url}' target='_blank'>{record.url}</a></div>"

            html += "</div>"  # End transcript block

        except Exception as e:
            html += f"<div class='twilio-value'>Error fetching operator results: {escape(str(e))}</div></div>"

    html += "</div>"  # twilio-box

    display(HTML(html))


In [62]:
from datetime import datetime, timedelta
from IPython.display import display, HTML

# Twilio SIGNAL-style CSS block (no emojis)
style = """
<style>
.twilio-box {
    background: #0B0D17;
    border: 1px solid #661847;
    border-radius: 12px;
    color: #ffffff;
    font-family: 'Segoe UI', sans-serif;
    padding: 20px;
    margin: 20px auto;
    width: 95%;
    box-shadow: 0 0 20px rgba(242, 47, 70, 0.2);
}
.twilio-header {
    font-size: 1.1em;
    font-weight: bold;
    margin-bottom: 10px;
    color: #F22F46;
}
.twilio-sub {
    color: #0DC5FB;
    font-size: 0.9em;
    margin-bottom: 8px;
}
.twilio-record {
    background: #12284C;
    padding: 12px;
    border-radius: 8px;
    margin-bottom: 10px;
    border-left: 4px solid #4B0082;
    font-size: 0.9em;
}
.twilio-key {
    font-weight: bold;
    color: #FFFFFF;
}
.twilio-value {
    color: #BBBBBB;
}
</style>
"""

# Initialize last_update_date
if 'last_update_date' not in globals():
    last_update_date = datetime.now() - timedelta(days=1)

last_update_time = last_update_date.strftime('%Y-%m-%dT%H:%M:%SZ')

# Fetch transcripts
transcripts = client.intelligence.v2.transcripts.list(
    limit=100,
    after_date_created=last_update_time,
    after_start_time=last_update_time,
)

# Build HTML output
html = f"""
{style}
<div class="twilio-box">
  <div class="twilio-header">Twilio Conversational Intelligence</div>
  <div class="twilio-sub">Fetching transcripts after: <code>{last_update_time}</code></div>
  <div class="twilio-sub">Found <strong>{len(transcripts)}</strong> new transcripts:</div>
"""

for record in transcripts:
    html += '<div class="twilio-record">'
    for key, value in vars(record).items():
        html += f"<div><span class='twilio-key'>{key}:</span> <span class='twilio-value'>{value}</span></div>"
    html += '</div>'

# Update the timestamp for next run
last_update_date = datetime.now()
html += f"""
  <div class="twilio-sub">Updated last_update_date to: <code>{last_update_date.strftime('%Y-%m-%d %H:%M:%S')}</code></div>
</div>
"""

# Display in notebook
display(HTML(html))


___

___



# Twilio Brasil - Signal São Paulo 2025



___

In [63]:
import time
from IPython.display import display, clear_output, HTML

# Messages to cycle through
messages = [
    "Welcome, Builders!",
    "Bienvenidos, Creadores!",
    "Bem-vindos, Criadores!",
    "Bienvenidas, Creadoras!",
    "Bem-vindas, Criadoras!"
]

# Final message
final_message = "TWILIO SIGNAL SÃO PAULO 2025"

# Shared CSS for animated gradient box
style = """
<style>
@keyframes gradient {
    0% { background-position: 0% 50%; }
    50% { background-position: 100% 50%; }
    100% { background-position: 0% 50%; }
}
.welcome-box {
    font-family: 'Segoe UI', sans-serif;
    font-size: 2em;
    font-weight: bold;
    color: white;
    padding: 30px;
    text-align: center;
    border-radius: 16px;
    background: linear-gradient(-45deg, #0DC5FB, #661847, #4B0082, #F22F46);
    background-size: 400% 400%;
    animation: gradient 8s ease infinite;
    box-shadow: 0 0 20px rgba(0,0,0,0.4);
    margin: 40px auto;
    width: 70%;
}
</style>
"""

# Animation loop
for _ in range(2):  # Number of cycles
    for msg in messages:
        clear_output(wait=True)
        html = f"{style}<div class='welcome-box'>{msg}</div>"
        display(HTML(html))
        time.sleep(1.5)

# Final SIGNAL banner
clear_output(wait=True)
html = f"{style}<div class='welcome-box'>{final_message}</div>"
display(HTML(html))


___

### Melhores Práticas para Escalar o <br>Engajamento de Clientes Multi-Agente <br>com Conversation Relay

**Content**


1. **[Voice API]()**  *Orquestração de voz*

2. **[TwiML]()**  *Controles de chamada avançados*

3. **[Connect:]()**   *ConversationRelay*

4. **[Conversational Intelligence]()**   *Observabilidade nativa*

___

<style>
.fvm-card {
  background-color: #0B0D17;
  color: #ffffff;
  border: 1px solid #661847;
  border-radius: 16px;
  padding: 24px;
  width: fit-content;
  max-width: 500px;
  font-family: 'Segoe UI', sans-serif;
  box-shadow: 0 0 20px rgba(242, 47, 70, 0.15);
  display: flex;
  align-items: center;
  gap: 20px;
  margin: 20px auto;
  min-height: fit-content;
  overflow: hidden;
}
.fvm-card img {
  width: 84px;
  height: 84px;
  border-radius: 50%;
  border: 2px solid #F22F46;
  object-fit: cover;
  flex-shrink: 0;
}
.fvm-card .info {
  display: flex;
  flex-direction: column;
  justify-content: center;
  line-height: 1.4;
  flex: 1;
  min-height: 0;
}
.fvm-card .name {
  font-size: 1.1em;
  font-weight: 600;
  color: #F22F46;
  margin: 0 0 4px 0;
}
.fvm-card .title {
  color: #0DC5FB;
  font-size: 0.95em;
  margin: 0;
}
</style>

<div class="fvm-card">
  <img src="https://assets.swoogo.com/uploads/medium/5396236-683722fd8921b.jpeg" alt="Fernando Vieira Machado">
  <div class="info">
    <div class="name">Fernando Vieira Machado</div>
    <div class="title">Arquiteto de Soluções</div>
    <div class="title">Twilio</div>
  </div>
</div>


## Architectural Overview

In [64]:
from IPython.display import display
from mermaid import Mermaid

mermaid_string = """
%%{init: {
  "theme": "base",
  "themeVariables": {
    "background": "#0B0D17",
    "primaryColor": "#12284C",
    "primaryBorderColor": "#661847",
    "primaryTextColor": "#FFFFFF",
    "tertiaryColor": "#0DC5FB",
    "tertiaryBorderColor": "#0B3D91",
    "tertiaryTextColor": "#FFFFFF",
    "noteBkgColor": "#4B0082",
    "noteTextColor": "#FFFFFF",
    "clusterBkg": "#0B0D17",
    "clusterBorder": "#1D1F2B"
  },
  "flowchart": {
    "useMaxWidth": true,
    "htmlLabels": true,
    "wrap": true
  }
}}%%

graph TD

    %% Step Labels - placed left of clusters
    Step1Label[/"Step 1: Inbound / Outbound Call"/]:::steplabel
    Step2Label[/"Step 2: Twilio Platform"/]:::steplabel
    Step3Label[/"Step 3: External AI Service"/]:::steplabel

    Step1Label --> PSTN
    Step2Label --> VoiceAPI
    Step3Label --> ExtService

    %% Step 1: Channels
    subgraph A[" "]
        PSTN["PSTN"]
        SIP["SIP"]
        WhatsApp["WhatsApp"]
        WebRTC["WebRTC / In-App"]
    end

    %% Step 2: Twilio Core
    subgraph B[" "]
        VoiceAPI["Voice API"]
        subgraph Orchestration["Orchestration Layer"]
            ConvRelay["Conversation Relay"]
            Studio["Studio"]
            TwiML["TwiML Bins"]
            Functions["Functions"]
            Webhooks["External Webhooks"]
        end
    end

    %% Step 3: External AI
    subgraph C[" "]
        ExtService["Relay Backend"]
        subgraph LLMs["LLM Models"]
            GPT["GPT"]
            Gemini["Gemini"]
            Claude["Claude"]
            DeepSeek["DeepSeek"]
        end
    end

    %% Connections
    PSTN --> VoiceAPI
    SIP --> VoiceAPI
    WhatsApp --> VoiceAPI
    WebRTC --> VoiceAPI

    VoiceAPI --> Studio
    VoiceAPI --> TwiML
    VoiceAPI --> Functions
    VoiceAPI --> Webhooks
    VoiceAPI --> ConvRelay

    ConvRelay -.->|"STT + Events"| ExtService
    ExtService -.->|"TTS Response"| ConvRelay

    ExtService --> GPT
    ExtService --> Gemini
    ExtService --> Claude
    ExtService --> DeepSeek

    %% Styling
    classDef entrypoint fill:#0DC5FB,stroke:#0B3D91,stroke-width:2px,color:#ffffff
    classDef twilio fill:#12284C,stroke:#661847,stroke-width:2px,color:#ffffff
    classDef relay fill:#F22F46,stroke:#0B3D91,stroke-width:2px,color:#ffffff
    classDef backend fill:#4B0082,stroke:#F22F46,stroke-width:2px,color:#ffffff
    classDef llm fill:#ffffff,stroke:#4B0082,stroke-width:2px,color:#4B0082
    classDef steplabel fill:#1D1F2B,stroke:#333333,stroke-width:1px,color:#ffffff

    class PSTN,SIP,WhatsApp,WebRTC entrypoint
    class VoiceAPI,Studio,TwiML,Functions,Webhooks twilio
    class ConvRelay relay
    class ExtService backend
    class GPT,Gemini,Claude,DeepSeek llm
"""

display(Mermaid(mermaid_string))


___

## Twilio Voice API

**Realizar uma chamada**

In [65]:
call = client.calls.create(
    from_="+551150397615",
    to="+5511968432422",
    twiml="<Response> <Say> Ahoy! Signal São Paulo! </Say> </Response>",
)

animate_call_flow_html(call.sid)


## TwiML 101

**Controle de Chamadas com a Twilio Markup Language**

In [80]:
from IPython.display import display
from mermaid import Mermaid

say_diagram = """
%%{init: {
  "theme": "base",
  "themeVariables": {
    "background": "#0B0D17",
    "primaryColor": "#12284C",
    "primaryBorderColor": "#661847",
    "primaryTextColor": "#FFFFFF",
    "tertiaryColor": "#0DC5FB",
    "tertiaryBorderColor": "#0B3D91",
    "tertiaryTextColor": "#FFFFFF",
    "noteBkgColor": "#4B0082",
    "noteTextColor": "#FFFFFF",
    "clusterBkg": "#0B0D17",
    "clusterBorder": "#1D1F2B"
  }
}}%%

sequenceDiagram
    participant App as Seu App
    participant Twilio as Twilio Voice API
    participant User as Usuário Final

    Note over App,User: Outbound Call
    App->>Twilio: Inicia chamada
    Twilio->>User: Conecta chamada
    Twilio->>App: Solicita TwiML
    App-->>Twilio: <Say>Mensagem</Say>
    Twilio-->>User: Vocaliza mensagem

    Note over User,App: Inbound Call
    User->>Twilio: Liga para número
    Twilio->>App: Solicita TwiML
    App-->>Twilio: <Say>Mensagem</Say>
    Twilio-->>User: Vocaliza mensagem
"""

display(Mermaid(say_diagram))


#### TwiML `<Say>` 
Text to Speech (TTS)

In [ ]:
# SAY

<Response>
  <Say language="fr-FR">Bonjour!</Say>
</Response>

In [ ]:
twiml_gather = """
    sequenceDiagram
    participant App as Seu App
    participant Twilio as Twilio Voice API
    participant User as Usuário Final

    Note over App,User: Outbound Call
    App->>Twilio: Inicia chamada (API Call)
    Twilio->>User: Conecta chamada
    Twilio->>App: Solicita instruções TwiML
    App-->>Twilio: <Gather>...</Gather>
    Twilio-->>User: Reproduz prompt e aguarda resposta
    User-->>Twilio: Responde (fala ou DTMF)
    Twilio-->>App: Envia dados capturados (STT/DTMF)

    Note over User,App: Inbound Call
    User->>Twilio: Liga para seu número
    Twilio->>App: Solicita instruções TwiML
    App-->>Twilio: <Gather>...</Gather>
    Twilio-->>User: Reproduz prompt e aguarda resposta
    User-->>Twilio: Responde (fala ou DTMF)
    Twilio-->>App: Envia dados capturados (STT/DTMF)
"""

#### TwiML `<Gather>` 

** Real-time transcription (speech to text  or STT) and digits input (DTMF)

In [ ]:
# GATHER

<Response>

  <Gather input="speech dtmf" finishOnKey="#" timeout="5">
    <Say>
      Please say something or press * to access the main menu
    </Say>
  </Gather>

  <Say>We didn't receive any input. Goodbye!</Say>
  
</Response>

In [81]:
gather_diagram = """
%%{init: {
  "theme": "base",
  "themeVariables": {
    "background": "#0B0D17",
    "primaryColor": "#12284C",
    "primaryBorderColor": "#661847",
    "primaryTextColor": "#FFFFFF",
    "tertiaryColor": "#0DC5FB",
    "tertiaryBorderColor": "#0B3D91",
    "tertiaryTextColor": "#FFFFFF",
    "noteBkgColor": "#4B0082",
    "noteTextColor": "#FFFFFF",
    "clusterBkg": "#0B0D17",
    "clusterBorder": "#1D1F2B"
  }
}}%%

sequenceDiagram
    participant App as Seu App
    participant Twilio as Twilio Voice API
    participant User as Usuário Final

    Note over App,User: Outbound Call
    App->>Twilio: Inicia chamada
    Twilio->>User: Conecta chamada
    Twilio->>App: Solicita TwiML
    App-->>Twilio: <Gather>...</Gather>
    Twilio-->>User: Lê prompt e aguarda entrada
    User-->>Twilio: Responde (fala ou DTMF)
    Twilio-->>App: Retorna dados capturados

    Note over User,App: Inbound Call
    User->>Twilio: Liga para número
    Twilio->>App: Solicita TwiML
    App-->>Twilio: <Gather>...</Gather>
    Twilio-->>User: Lê prompt e aguarda entrada
    User-->>Twilio: Responde (fala ou DTMF)
    Twilio-->>App: Retorna dados capturados
"""

display(Mermaid(gather_diagram))


#### Usando TwiML em Python com a lib da Twilio 

**`<Say>` = response.`say`**

In [67]:
## SAY usando a biblioteca Twilio

from twilio.twiml.voice_response import VoiceResponse

response = VoiceResponse()

response.say(
    language="fr-FR", 
    text="Bonjour!"
    )

pretty_print_twiml(response)

'<Response>\n  <Say language="fr-FR" text="Bonjour!"/>\n</Response>'

**`<Gather>`= response.`gather`**

In [68]:
# GATHER usando a biblioteca Twilio

from twilio.twiml.voice_response import Gather, VoiceResponse, Say

response = VoiceResponse()
gather = Gather(action='/process_gather.php', method='GET')

gather.say('Please enter your account number,\nfollowed by the pound sign')
response.append(gather)

response.say('We didn\'t receive any input. Goodbye!')

import xml.dom.minidom

pretty_print_twiml(response)

'<Response>\n  <Gather action="/process_gather.php" method="GET">\n    <Say>Please enter your account number,\nfollowed by the pound sign</Say>\n  </Gather>\n  <Say>We didn\'t receive any input. Goodbye!</Say>\n</Response>'

### Realizando uma chamada com TwiML Say e Gather

In [69]:
# Fazendo uma chamada usando SAY e GATHER
from twilio.twiml.voice_response import VoiceResponse, Gather

response = VoiceResponse()

# Gather DTMF or speech
gather = Gather(input="speech dtmf", timeout=5, action="/handle_input", method="POST") #TO DO: Add route to handle input
gather.say("Por favor, me fale como posso ajudar você hoje. Ou digite um número qualquer seguido de cerquilha.")
response.append(gather)

# TO DO: Add route to handle input in server

# Fallback if no input
response.say("Poxa! Não recebemos nenhuma resposta. Tchau tchau!")

pretty_print_twiml(str(response))  # optional: view generated XML

# Make the call
call = client.calls.create(
    from_="+551150397615",
    to="+5511968432422",
    twiml=str(response)
)

print(call.sid)


CA62cf2f298d60110dd253e219c0214032


___

## Twilio Conversation Relay

### Conectando chamadas com o TwiML `<Connect>`

In [ ]:
<?xml version="1.0" encoding="UTF-8"?>

<Response>

  <Connect action="https://my-https-server.com/connect_action">
    <ConversationRelay url="wss://my-https-websocket-server.com/websocket" welcomeGreeting="Hi! Ask me anything!" />
  </Connect>

</Response>

In [ ]:
from IPython.display import display
from mermaid import Mermaid

mermaid_string = """
%%{init: {
    "theme": "base",
    "themeVariables": {
        "background": "#0B0D17",
        "primaryColor": "#12284C",
        "primaryBorderColor": "#661847",
        "primaryTextColor": "#FFFFFF",
        "tertiaryColor": "#0DC5FB",
        "tertiaryBorderColor": "#0B3D91",
        "tertiaryTextColor": "#FFFFFF",
        "noteBkgColor": "#4B0082",
        "noteTextColor": "#FFFFFF",
        "clusterBkg": "#0B0D17",
        "clusterBorder": "#1D1F2B"
    },
    "flowchart": {
        "useMaxWidth": true,
        "htmlLabels": true,
        "wrap": true
    }
}}%%

graph TD

        %% Voice Call at top
        VoiceCall["Voice Call<br/>(Inbound/Outbound)"]:::voice

        %% AI Components on the left
        RAG["RAG<br/>(Tools | Knowledge)"]:::ai
        MCP["MCP<br/>(Model Context<br/>Protocol)"]:::ai

        %% Conversation Relay in center
        ConvRelay["Conversation Relay"]:::relay

        %% LLM Models on the right
        GPT["GPT"]:::llm
        Gemini["Gemini"]:::llm
        Claude["Claude"]:::llm
        DeepSeek["DeepSeek"]:::llm

        %% External Server at bottom
        ExtServer["External Server<br/>(WebSocket Handler)"]:::backend

        %% Vertical main flow
        VoiceCall -->|"Customer Speech"| ConvRelay
        ConvRelay -->|"STT | Events"| ExtServer

        %% Horizontal connections to AI components
        ExtServer -.-> RAG
        ExtServer -.-> MCP
        
        %% Connections to LLM models
        ExtServer -->|"Text Prompts"| GPT
        ExtServer -->|"Text Prompts"| Gemini
        ExtServer -->|"Text Prompts"| Claude
        ExtServer -->|"Text Prompts"| DeepSeek

        %% Return flow
        GPT -->|"AI Response"| ExtServer
        Gemini -->|"AI Response"| ExtServer
        Claude -->|"AI Response"| ExtServer
        DeepSeek -->|"AI Response"| ExtServer

        RAG -.-> ExtServer
        MCP -.-> ExtServer

        ExtServer -->|"LLM Response"| ConvRelay
        ConvRelay -->|"TTS<br/>(Text to Speech)"| VoiceCall

        %% Styling
        classDef voice fill:#0DC5FB,stroke:#0B3D91,stroke-width:3px,color:#ffffff
        classDef relay fill:#F22F46,stroke:#0B3D91,stroke-width:3px,color:#ffffff
        classDef backend fill:#4B0082,stroke:#F22F46,stroke-width:3px,color:#ffffff
        classDef ai fill:#661847,stroke:#F22F46,stroke-width:2px,color:#ffffff
        classDef llm fill:#ffffff,stroke:#4B0082,stroke-width:2px,color:#4B0082
"""

display(Mermaid(mermaid_string))

In [83]:
from IPython.display import display
from mermaid import Mermaid

sequence_mermaid_simplified = """
%%{init: {
  "theme": "base",
  "themeVariables": {
    "background": "#0B0D17",
    "primaryColor": "#12284C",
    "primaryBorderColor": "#661847",
    "primaryTextColor": "#FFFFFF",
    "tertiaryColor": "#0DC5FB",
    "tertiaryBorderColor": "#0B3D91",
    "tertiaryTextColor": "#FFFFFF",
    "noteBkgColor": "#4B0082",
    "noteTextColor": "#FFFFFF",
    "clusterBkg": "#0B0D17",
    "clusterBorder": "#1D1F2B"
  }
}}%%

sequenceDiagram
    participant User as Voice Call (Usuário)
    participant Relay as Conversation Relay
    participant Server as External Server
    participant RAG as RAG (Ferramentas & Conhecimento)
    participant MCP as MCP (Model Context Protocol)
    participant LLM as LLM (GPT, Gemini, Claude, DeepSeek)

    User->>Relay: Fala do cliente (voz)
    Relay->>Server: STT + Eventos (via WebSocket)

    Server-->>RAG: Busca Ferramentas / Conhecimento
    Server-->>MCP: Ajusta contexto

    Server->>LLM: Prompt com texto
    LLM-->>Server: Resposta da IA

    Server->>Relay: Texto final da IA
    Relay->>User: Resposta (via TTS)
"""

display(Mermaid(sequence_mermaid_simplified))


### Conversation Relay usando a biblioteca da Twilio para Python

In [ ]:
from twilio.twiml.voice_response import Connect, ConversationRelay, Language, VoiceResponse

response = VoiceResponse()
connect = Connect()

conversationrelay = ConversationRelay(
    url='wss://my-websocket-server.com/websocket') # External Server integrated to the LLMs

conversationrelay.language(
    code='pt-BR',
    tts_provider='ElevenLabs', # Text to Speech provider
    voice='UgBBYS2sOqTuMpoF3BR0', # Voice ID from tts_provider ElevenLabs
    transcription_provider='google', 
    speech_model='telephony')

connect.append(conversationrelay)

response.append(connect)

print(response)

___

## _Quem sabe faz ao vivis_

### Fazendo chamadas com `<ConversationRelay>`

In [85]:
from twilio.twiml.voice_response import VoiceResponse, Connect, ConversationRelay

# Get your ngrok domain from .env file
ngrok_domain = os.getenv('NGROK_DOMAIN', 'your-custom-domain.ngrok-free.app')

# Create TwiML with Conversation Relay
response = VoiceResponse()
connect = Connect()

In [ ]:

# Create ConversationRelay pointing to your WebSocket server
conversation_relay = ConversationRelay(
    
    url=f'wss://{ngrok_domain}/websocket',

    welcome_greeting="Olá! Eu sou o assistente virtual do Sígnal São Paulo. Que evento lindo, hein? Como posso ajudar você hoje?",
    
    # Idioma da saudação e default da conversa
    language='pt-BR',

    # Motor de vocalização e voz
    tts_provider='ElevenLabs',
    voice='7u8qsX4HQsSHJ0f8xsQZ',

    # Motor de transcrição e modelo
    transcription_provider='Deepgram',
    speech_model='nova-2-general',
    
)

connect.append(conversation_relay)
response.append(connect)

# Make the call with Conversation Relay
call = client.calls.create(
    from_="+551150397615",
    to="+5511968432422",
    twiml=str(response)
)

print(f"Call SID: {call.sid}")
print(f"WebSocket URL: wss://{ngrok_domain}")
pretty_print_twiml(response)

# open terminal for tts, stt events ;)

Call SID: CA8c701200140f3bb2aa6fbbb49b965fa2
WebSocket URL: wss://owlbank.ngrok.io


'<Response>\n  <Connect>\n    <ConversationRelay language="pt-BR" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" url="wss://owlbank.ngrok.io/websocket" voice="7u8qsX4HQsSHJ0f8xsQZ" welcomeGreeting="Olá! Eu sou o assistente virtual do Sígnal São Paulo. Que evento lindo, hein? Como posso ajudar você hoje?"/>\n  </Connect>\n</Response>'

## Aprimorando a experiência com `attributes` 

**Vamos melhorar isso um pouco?**

In [ ]:
# Aprimorando as chamadas feitas com Conversation Relay

conversation_relay = ConversationRelay(
    url=f'wss://{ngrok_domain}/websocket',


    welcome_greeting="Olá! Eu sou o assistente virtual do Sígnal São Paulo. Que evento lindo, hein? Como posso ajudar você hoje?",
    welcomeGreetingInterruptible=True,  # Permitir que o usuário interrompa a saudação


    language='pt-BR',    
    transcription_provider='Deepgram',
    speech_model='nova-2-general',    
    tts_provider='ElevenLabs',
    voice='7u8qsX4HQsSHJ0f8xsQZ',

    # Configurações adicionais para aprimorar a experiência do usuário
    interruptible=True,  # Permitir que o usuário interrompa o assistente

    dtmfDetection=True,  # Habilitar detecção de DTMF (dígitos)

    reportInputDuringAgentSpeech=True,  # Reportar entradas do usuário durante o discurso do agente

    preemptible=True,  # Permitir que novos tokens do assistente interrompam o TTS em andamento

    hints='Twilio, Conversation Relay, Signal São Paulo, Owl Bank', # Sugestões para melhorar a precisão do reconhecimento de fala

    debug='debugging, speaker-events, tokens-played', # opções de depuração, eventos de debug, fala e tokens tocados

    elevenlabsTextNormalization='on', # Normalização de texto para ElevenLabs TTS
)

# Adicionando o idioma e o modelo de fala
conversation_relay.language(
    code='pt-BR',
    tts_provider='ElevenLabs',
    voice='7u8qsX4HQsSHJ0f8xsQZ',
    transcription_provider='Deepgram',
    speech_model='nova-2-general'
)

connect.append(conversation_relay)
response.append(connect)

# Make the call with Conversation Relay
call = client.calls.create(
    from_="+551150397615",
    to="+5511968432422",
    twiml=str(response)
)

print(f"Call SID: {call.sid}")
print(f"WebSocket URL: wss://{ngrok_domain}")
pretty_print_twiml(response)

## Como Escalar com Conversation Relay?

### `<Language>` 

Nativamente Multi-Idiomas, Flexibilidade de Modelos para TTS e STT

In [37]:
from twilio.twiml.voice_response import VoiceResponse, Connect, ConversationRelay

# Get your ngrok domain from .env file
ngrok_domain = os.getenv('NGROK_DOMAIN', 'your-custom-domain.ngrok-free.app')

# Create TwiML with Conversation Relay
response = VoiceResponse()
connect = Connect()

In [38]:
# Suporte nativo a múltiplos idiomas

conversation_relay = ConversationRelay(
    url=f'wss://{ngrok_domain}/websocket',
    welcome_greeting="Oi de novo! Aqui é a assistente virtual do Sígnal São Paulo. Como posso te ajudar?",
    welcomeGreetingInterruptible=True,  # Permitir que o usuário interrompa a saudação
    language='pt-BR',
    debug='debugging, speaker-events, tokens-played', # opções de depuração
    interruptible=True,  # Permitir que o usuário interrompa o assistente
    dtmfDetection=True,  # Habilitar detecção de DTMF (dígitos)
    reportInputDuringAgentSpeech=True,  # Relatar entradas do usuário durante o discurso do agente
    preemptible=True,  # Permitir que novos tokens do assistente interrompam o TTS em andamento
    hints='Twilio, Conversation Relay, Signal São Paulo, Owl Bank',
    elevenlabsTextNormalization='on',
)

# Configurar idioma e modelo de fala para português do Brasil
conversation_relay.language(
    code='pt-BR',
    tts_provider='ElevenLabs',
    voice='OB6x7EbXYlhG4DDTB1XU',
    transcription_provider='Deepgram',
    speech_model='nova-2-general',
    customParameter='change_to_pt-BR'
)

# Configurar idioma e modelo de fala para espanhol dos EUA
conversation_relay.language(
    code='es-US',
    tts_provider='ElevenLabs',
    voice='OB6x7EbXYlhG4DDTB1XU',
    transcription_provider='Deepgram',
    speech_model='nova-2-general',
    customParameter='change_to_es-US'
)

# Configurar idioma e modelo de fala para inglês dos EUA
conversation_relay.language(
    code='en-US',
    tts_provider='ElevenLabs',
    voice='OB6x7EbXYlhG4DDTB1XU',
    transcription_provider='Deepgram',
    speech_model='nova-2-general',
    customParameter="change_to_en-US"
)

connect.append(conversation_relay)
response.append(connect)

# Make the call with Conversation Relay
call = client.calls.create(
    from_="+551150397615",
    to="+5511968432422",
    twiml=str(response)
)

print(f"Call SID: {call.sid}")
print(f"WebSocket URL: wss://{ngrok_domain}")
pretty_print_twiml(response)

Call SID: CA12c2eb6b966bc564ada4631247442938
WebSocket URL: wss://owlbank.ngrok.io


'<Response>\n  <Connect>\n    <ConversationRelay debug="debugging, speaker-events, tokens-played" dtmfDetection="true" elevenlabsTextNormalization="on" hints="Twilio, Conversation Relay, Signal São Paulo, Owl Bank" interruptible="true" language="pt-BR" preemptible="true" reportInputDuringAgentSpeech="true" url="wss://owlbank.ngrok.io/websocket" welcomeGreeting="Oi de novo! Aqui é a assistente virtual do Sígnal São Paulo. Como posso te ajudar?" welcomeGreetingInterruptible="true">\n      <Language code="pt-BR" customParameter="change_to_pt-BR" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" voice="OB6x7EbXYlhG4DDTB1XU"/>\n      <Language code="es-US" customParameter="change_to_es-US" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" voice="OB6x7EbXYlhG4DDTB1XU"/>\n      <Language code="en-US" customParameter="change_to_en-US" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" 

### Engajamento de Clientes Multi-Agente

**Aprimorando a experiência com agentes especialistas**

In [ ]:
# TO DO: simplify mas code for notebook

### Observabiidade com Conversational Intelligence

In [56]:
# Aprimorando as chamadas feitas com Conversation Relay

from twilio.twiml.voice_response import VoiceResponse, Connect, ConversationRelay

# Get your ngrok domain from .env file
ngrok_domain = os.getenv('NGROK_DOMAIN', 'your-custom-domain.ngrok-free.app')

# Create TwiML with Conversation Relay
response = VoiceResponse()
connect = Connect()

# Create ConversationRelay pointing to your WebSocket server
conversation_relay = ConversationRelay(
    url=f'wss://{ngrok_domain}/websocket',
    welcome_greeting="Tarde! Assistente virtual do Signal São Paulo na área. Como posso te ajudar, chefia?",
    welcomeGreetingInterruptible=True,  # Allow user to interrupt the welcome greeting
    language='pt-BR',
    transcription_provider='Deepgram',
    speech_model='nova-2-general',
    tts_provider='ElevenLabs',
    voice='eQnBc1norhy4xHHbr9Ip',
    interruptible=True,  # Allow user to interrupt the assistant
    dtmfDetection=True,  # Enable DTMF detection
    reportInputDuringAgentSpeech=True,  # Report input during agent speech
    preemptible=True,  # Allow the assistant to preempt the user
    hints='Twilio, Conversation Relay, Signal São Paulo, Owl Bank',
    debug='debugging, speaker-events, tokens-played', # debbugging options
    elevenlabsTextNormalization='on',
    intelligenceService='GAde9c513fd3914897cac25df18f3203b7'
)

# Configure language settings for Brazilian Portuguese
conversation_relay.language(
    code='pt-BR',
    tts_provider='ElevenLabs',
    voice='7u8qsX4HQsSHJ0f8xsQZ',
    transcription_provider='Deepgram',
    speech_model='nova-2-general'
)

connect.append(conversation_relay)
response.append(connect)

# Make the call with Conversation Relay
call = client.calls.create(
    from_="+551150397615",
    to="+5511968432422",
    twiml=str(response)
)

print(f"Call SID: {call.sid}")
print(f"WebSocket URL: wss://{ngrok_domain}")
pretty_print_twiml(response)

Call SID: CA05db167b904baa44fc854efdcabebdcb
WebSocket URL: wss://owlbank.ngrok.io


'<Response>\n  <Connect>\n    <ConversationRelay debug="debugging, speaker-events, tokens-played" dtmfDetection="true" elevenlabsTextNormalization="on" hints="Twilio, Conversation Relay, Signal São Paulo, Owl Bank" intelligenceService="GAde9c513fd3914897cac25df18f3203b7" interruptible="true" language="pt-BR" preemptible="true" reportInputDuringAgentSpeech="true" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" url="wss://owlbank.ngrok.io/websocket" voice="eQnBc1norhy4xHHbr9Ip" welcomeGreeting="Tarde! Assistente virtual do Signal São Paulo na área. Como posso te ajudar, chefia?" welcomeGreetingInterruptible="true">\n      <Language code="pt-BR" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" voice="7u8qsX4HQsSHJ0f8xsQZ"/>\n    </ConversationRelay>\n  </Connect>\n</Response>'

### Verificando os resultados com Conversational Intelligence

In [52]:
# List Transcripts Incrementally
from datetime import datetime

# Initialize or use existing last_update_date
if 'last_update_date' not in globals():
    # First run - get transcripts from last 24 hours to avoid missing recent ones
    from datetime import timedelta
    last_update_date = datetime.now() - timedelta(days=1)

last_update_time = last_update_date.strftime('%Y-%m-%dT%H:%M:%SZ')
print(f"Fetching transcripts after: {last_update_time}")

transcripts = client.intelligence.v2.transcripts.list(
    limit=100,
    after_date_created=last_update_time,
    after_start_time=last_update_time,
)

print(f"Found {len(transcripts)} new transcripts:\n")

for record in transcripts:
    for key, value in vars(record).items():
        print(f"{key}: {value}")
    print("\n---\n")

# Update the timestamp for next run
last_update_date = datetime.now()
print(f"Updated last_update_date to: {last_update_date}")

Fetching transcripts after: 2025-06-22T10:23:58Z
Found 11 new transcripts:

_version: <Twilio.Intelligence.V2>
account_sid: ACdf269d13302fe4635ab695fb1b477395
service_sid: GAde9c513fd3914897cac25df18f3203b7
sid: GT92b8758f818f48f099cc5f68588d59e7
date_created: 2025-06-22 15:49:26+00:00
date_updated: 2025-06-22 15:49:33+00:00
status: completed
channel: {'media_properties': {'source': 'ConversationRelay', 'reference_sids': {'call_sid': 'CAc8f2e7261c455caea60b5e2160b855b1'}, 'source_sid': 'VXff79704b3932752b9e1c10164e72c312', 'media_url': None}, 'participants': [{'user_id': None, 'channel_participant': 1, 'media_participant_id': 'Virtual Agent', 'image_url': None, 'full_name': 'Virtual Agent', 'role': 'Virtual Agent', 'email': None}, {'user_id': None, 'channel_participant': 2, 'media_participant_id': '+551150397615', 'image_url': None, 'full_name': '+551150397615', 'role': 'Customer', 'email': None}], 'type': 'voice'}
data_logging: False
language_code: pt-BR
customer_key: None
media_start

In [53]:
# List Operator Results for all Transcripts found incrementally
print(f"Getting operator results for {len(transcripts)} transcripts...\n")

for transcript_record in transcripts:
    print(f"TRANSCRIPT: {transcript_record.sid}")
    print("=" * 80)
    
    try:
        operator_results = client.intelligence.v2.transcripts(
            transcript_record.sid
        ).operator_results.list(limit=100)
        
        print(f"Found {len(operator_results)} operator results:\n")

        for i, record in enumerate(operator_results, 1):
            print(f"OPERATOR RESULT #{i}")
            print("=" * 60)
            print(f"Name: {record.name}")
            print(f"Type: {record.operator_type}")
            print(f"Operator SID: {record.operator_sid}")
            
            # Handle different result types
            if record.operator_type == 'text-generation' and record.text_generation_results:
                print(f"\nText Generation Result:")
                print("-" * 30)
                result_text = record.text_generation_results.get('result', 'No result available')
                print(f"{result_text}")
                
            elif record.operator_type == 'extract' and record.extract_results:
                print(f"\nExtraction Results:")
                print("-" * 30)
                for entity_type, entities in record.extract_results.items():
                    print(f"  {entity_type}: {entities}")
                
                if record.extract_match:
                    print(f"\nMatch Found: {record.extract_match}")
                    print(f"Match Probability: {record.match_probability}")
                    
                if record.utterance_results:
                    print(f"\nUtterance Analysis:")
                    for j, utterance in enumerate(record.utterance_results):
                        print(f"  Utterance {j+1} (Index {utterance['utterance_index']}):")
                        print(f"    Match Probability: {utterance['match_probability']}")
                        labeled_text = ""
                        for part in utterance['utterance_parts']:
                            if part['label']:
                                labeled_text += f"[{part['label']}: {part['text']}]"
                            else:
                                labeled_text += part['text']
                        print(f"    Text: {labeled_text}")
                
            elif record.operator_type == 'conversation-classify':
                print(f"\nClassification Result:")
                print("-" * 30)
                print(f"Predicted Label: {record.predicted_label}")
                print(f"Predicted Probability: {record.predicted_probability}")
                
                if record.label_probabilities:
                    print(f"All Label Probabilities:")
                    for label, prob in record.label_probabilities.items():
                        print(f"  {label}: {prob}")
            
            print(f"\nTranscript SID: {record.transcript_sid}")
            print(f"URL: {record.url}")
            print("=" * 60)
            print()
            
    except Exception as e:
        print(f"Error fetching operator results for transcript {transcript_record.sid}: {e}")
    
    print("=" * 80)
    print()

Getting operator results for 11 transcripts...

TRANSCRIPT: GT92b8758f818f48f099cc5f68588d59e7
Found 11 operator results:

OPERATOR RESULT #1
Name: Virtual Agent Performance
Type: text-generation
Operator SID: LY69843e95cea04823b7d80cd08942978a

Text Generation Result:
------------------------------
1) The virtual agent was somewhat consultative in its approach, as it greeted the customer and acknowledged their request for information about the bank. However, it did not ask any follow-up questions to further understand the customer's specific needs or interests regarding banking services. For instance, it could have inquired whether the customer was looking for personal banking, business accounts, loans, or investment options, which would have allowed for a more tailored response.

2) The virtual agent provided a basic introduction to Owl Bank but did not delve into specific products or services that might meet the customer's needs. Given the lack of follow-up questions and the general

In [ ]:
# TO DO: Overview of scaling diagram

## Wrap-Up

**Melhores Práticas para Escalar o
<br>
Engajamento de Clientes Multi-Agentes 
<br>
com Conversation Relay**

&emsp; &#9658; Orquestração Omnicanal
<br>
&emsp; &#9658; Personalização em Tempo Real
<br>
&emsp; &#9658; Especialização de Agentes e Recursos
<br>
&emsp; &#9658; Governança e Observabilidade

#### Next Session:

**Conversas em escala: <br>
A nova era do engajamento pelo WhatsApp**

In [ ]:
import time
from IPython.display import display, clear_output, HTML

# Messages to cycle through
messages = [
    "Thank you!",
    "¡Gracias!",
    "Obrigado!"
]

# Final message
final_message = "TWILIO SIGNAL SÃO PAULO 2025"

# Shared CSS for animated gradient box
style = """
<style>
@keyframes gradient {
    0% { background-position: 0% 50%; }
    50% { background-position: 100% 50%; }
    100% { background-position: 0% 50%; }
}
.welcome-box {
    font-family: 'Segoe UI', sans-serif;
    font-size: 2em;
    font-weight: bold;
    color: white;
    padding: 30px;
    text-align: center;
    border-radius: 16px;
    background: linear-gradient(-45deg, #0DC5FB, #661847, #4B0082, #F22F46);
    background-size: 400% 400%;
    animation: gradient 8s ease infinite;
    box-shadow: 0 0 20px rgba(0,0,0,0.4);
    margin: 40px auto;
    width: 70%;
}
</style>
"""

# Animation loop
for _ in range(2):  # Number of cycles
    for msg in messages:
        clear_output(wait=True)
        html = f"{style}<div class='welcome-box'>{msg}</div>"
        display(HTML(html))
        time.sleep(1.5)

# Final SIGNAL banner
clear_output(wait=True)
html = f"{style}<div class='welcome-box'>{final_message}</div>"
display(HTML(html))
